In [1]:
import numpy as np
import seaborn as sns
from copy import deepcopy
import os
from pathlib import Path
from tqdm.auto import tqdm

/opt/anaconda3/envs/kaggle_tpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
root = Path("/home/edu/code/google_fast_or_slow/data/npz_all/npz")
collection = "layout/nlp"
ctype = "random"

In [7]:
def prune_graph(data):
    print("Pruning graph...")
    new_data = deepcopy(dict(data))
    print("Original graph has {} nodes and {} edges".format(data["node_feat"].shape[0], data["edge_index"].shape[0]))
    in_edge_index = data["edge_index"][np.isin(data["edge_index"], data["node_config_ids"]).any(1)]

    in_node_ids = np.unique(in_edge_index)
    assert len(set(data["node_config_ids"]) - set(in_node_ids)) == 0
    lookup = np.ones(data["node_feat"].shape[0]) * -1
    lookup[in_node_ids] = np.arange(in_node_ids.shape[0])

    in_node_feats = data["node_feat"][in_node_ids, :]
    in_node_opcode = data["node_opcode"][in_node_ids]
    in_edge_index = lookup[in_edge_index]
    in_node_config_ids = lookup[data["node_config_ids"]]

    new_data["node_feat"] = in_node_feats
    new_data["node_opcode"] = in_node_opcode
    new_data["edge_index"] = in_edge_index
    new_data["node_config_ids"] = in_node_config_ids
    print("New graph has {} nodes and {} edges".format(new_data["node_feat"].shape[0], new_data["edge_index"].shape[0]))
    return new_data


In [8]:
def remove_dupplicated_node_configs(data):
    reshaped_config_feat = data["node_config_feat"].reshape(data["node_config_feat"].shape[0], -1) + 2 # avoid zeros
    positional_array = np.random.random(reshaped_config_feat.shape[1])  # multiply each value by its position to avoid removing permutations by accident
    reshaped_values = (reshaped_config_feat * positional_array[None, :]).sum(1)
    is_equal_matrix = reshaped_values[None, :] == reshaped_values[:, None] # quadratic matrix of all pairwise equalities
    # is_equal_matrix[np.triu_indices(is_equal_matrix.shape[0], 0)] = 0 # only get diagonal to avoid remove twice
    is_equal_matrix = np.tril(is_equal_matrix, -1) # only get diagonal to avoid remove twice
    to_remove_ids = np.unique(np.where(is_equal_matrix)[0])
    print("Removing {} duplicated node configs out of {}".format(to_remove_ids.shape[0], data["node_config_feat"].shape[0]))
    data["config_runtime"] = np.delete(data["config_runtime"], to_remove_ids)
    data["node_config_feat"] = np.delete(data["node_config_feat"], to_remove_ids, axis=0)
    return data

In [9]:
dst_dir = root / f"{collection}_pruned" / ctype
for split in ["train", "valid", "test"]:
    print("Loading {} data...".format(split))
    split_src_dir = root / collection / ctype / split
    split_dst_dir = dst_dir / split
    split_dst_dir.mkdir(parents=True, exist_ok=True)

    for npz_path in tqdm(list(split_src_dir.glob("*.npz"))):
        print(npz_path)
        data = dict(np.load(str(npz_path), allow_pickle=True))
        data = prune_graph(data)
        if split == "train":
            data = remove_dupplicated_node_configs(data)
        np.savez(split_dst_dir / npz_path.name, **data)

Loading train data...


  0%|          | 0/207 [00:00<?, ?it/s]

/home/edu/code/google_fast_or_slow/data/npz_all/npz/layout/nlp/random/train/albert_en_xxlarge_batch_size_16_train.npz
Pruning graph...
Original graph has 5400 nodes and 9113 edges
New graph has 995 nodes and 1011 edges
Removing 39 duplicated node configs out of 24816


  0%|          | 1/207 [00:02<07:42,  2.25s/it]

/home/edu/code/google_fast_or_slow/data/npz_all/npz/layout/nlp/random/train/electra_base_batch_size_64_train.npz
Pruning graph...
Original graph has 10124 nodes and 16696 edges
New graph has 1134 nodes and 1077 edges
Removing 39 duplicated node configs out of 25016


...

Loading test data...


 35%|███▌      | 6/17 [00:00<00:00, 59.69it/s]

/home/edu/code/google_fast_or_slow/data/npz_all/npz/layout/nlp/random/test/492c7a94d559aa4a88769142d2a68362.npz
Pruning graph...
Original graph has 7292 nodes and 12004 edges
New graph has 818 nodes and 769 edges
/home/edu/code/google_fast_or_slow/data/npz_all/npz/layout/nlp/random/test/f6c146fc5cf10be4f3accbaca9897311.npz
Pruning graph...
Original graph has 5868 nodes and 9650 edges
New graph has 660 nodes and 615 edges
/home/edu/code/google_fast_or_slow/data/npz_all/npz/layout/nlp/random/test/d15316c12eefdef1ba549eb433797f77.npz
Pruning graph...
Original graph has 1536 nodes and 2332 edges
New graph has 291 nodes and 248 edges
/home/edu/code/google_fast_or_slow/data/npz_all/npz/layout/nlp/random/test/71b79ca6db513e7979c3702c595150c2.npz
Pruning graph...
Original graph has 3584 nodes and 5316 edges
New graph has 939 nodes and 848 edges
/home/edu/code/google_fast_or_slow/data/npz_all/npz/layout/nlp/random/test/29886a50d55cfe77a9497bc906c76ce9.npz
Pruning graph...
Original graph has 445

100%|██████████| 17/17 [00:00<00:00, 57.60it/s]

Pruning graph...
Original graph has 1096 nodes and 1688 edges
New graph has 183 nodes and 152 edges
/home/edu/code/google_fast_or_slow/data/npz_all/npz/layout/nlp/random/test/32531d07a084b319dce484f53a4cf3fc.npz
Pruning graph...
Original graph has 4453 nodes and 7305 edges
New graph has 502 nodes and 461 edges
/home/edu/code/google_fast_or_slow/data/npz_all/npz/layout/nlp/random/test/3a0c5517a87df8d82fd637b83298a3ba.npz
Pruning graph...
Original graph has 21918 nodes and 36278 edges
New graph has 2850 nodes and 2745 edges


In [ ]:
split_src_dir

In [3]:
path = "/home/edu/code/google_fast_or_slow/data/npz_all/npz/layout/xla/default/train/ncf.2x2.fp32.npz"
data = np.load(path)

In [4]:
list(data.keys())

['edge_index',
 'node_feat',
 'node_opcode',
 'node_config_feat',
 'node_config_ids',
 'node_splits',
 'config_runtime']

In [5]:
data["node_config_feat"].shape

(100040, 20, 18)

In [6]:
in_edge_index = data["edge_index"][np.isin(data["edge_index"], data["node_config_ids"]).any(1)]

in_node_ids = np.unique(in_edge_index)

In [10]:
len(set(data["node_config_ids"]) - set(in_node_ids))

0

In [ ]:
100040 - 99668

In [ ]:
reshaped_config_feat = data["node_config_feat"].reshape(data["node_config_feat"].shape[0], -1)
print(1)
positional_array = (np.arange(reshaped_config_feat.shape[1]) + 1) # multiply each value by its position to avoid removing permutations by accident
reshaped_values = (reshaped_config_feat * positional_array[None, :]).sum(1)
print(2)
is_equal_matrix = reshaped_values[None, :] == reshaped_values[:, None] # quadratic matrix of all pairwise equalities
print(3)
is_equal_matrix = np.tril(is_equal_matrix, -1) # only get diagonal to avoid remove twice
print(4)
to_remove_ids = np.unique(np.where(is_equal_matrix)[0])
print(5)
print("Removing {} duplicated node configs".format(to_remove_ids.shape[0]))

In [ ]:
remove_dupplicated_node_configs(data)

In [ ]:
(100040 ** 2) * 4 / 1e9

In [ ]:
remove_dupplicated_node_configs(dict(data))

In [ ]:
data["node_config_feat"].shape

In [ ]:
np.delete(x, to_remove_ids)


In [ ]:
is_equal_matrix.sum(1)

In [ ]:
x = np.array([1, 1, 2, 3, 3, 4, 7, 1, 2])
m = x[None, :] == x[:, None]
# remove 1,4, 7, 8

In [ ]:
m

In [ ]:
np.tril(m, -1)

In [ ]:

m

In [ ]:
to_remove_ids = np.unique(np.where(m)[0])

In [ ]:
np.delete(x, to_remove_ids)

In [ ]:
(5304 ** 2) / 1e9

In [ ]:
new_data = prune_graph(data)

In [ ]:
data["edge_index"].shape#[3, :]

In [ ]:

in_node_feats.shape

In [ ]:
data["node_feat"].shape

In [ ]:
np.unique(data["node_config_feat"][:, :, 98], return_counts=True)

In [ ]:
{k: data[k].shape for k in data.keys()}

In [ ]:
np.unique((data["node_config_feat"].sum(2) != -18).sum(1), return_counts=True)

In [ ]:
sns.histplot((data["node_config_feat"].sum(2) != -18).sum(1))

In [ ]:
((data["node_config_feat"][0] != data["node_config_feat"][3]).sum(1) > 0).sum()

In [ ]:
100040 * 100040 * 1 / 1e9

In [ ]:
from tqdm.auto import tqdm

In [ ]:
data["node_config_feat"][i:i+1].shape

In [ ]:
((data["node_config_feat"][i:i+1] != data["node_config_feat"]).sum(2) > 0).sum(1)

In [ ]:
max_size = 100
bz = 100
result = np.zeros((max_size, 100040), dtype=np.uint8)
for i in tqdm(range(0, max_size, bz)):
    result[i:i+bz] = ((data["node_config_feat"][i:i+bz, None] != data["node_config_feat"][None, ...]).sum(-1) > 0).sum(-1)

In [ ]:
result.max(1)

In [ ]:
(result == 1).sum(1)

In [ ]:
(result == 0).sum(1)

In [ ]:
result.min(1)

In [ ]:
np.median(result, 1)

In [ ]:
np.percentile(result, 0.1, 1)

In [ ]:
(data["node_config_feat"][0] != data["node_config_feat"][0][1]).sum()

In [ ]:
np.unique(data["node_feat"][:, 22])

In [1]:
import numpy as np
import torch

In [12]:
x = torch.arange(12).reshape(4, 3).float()
x

tensor([[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.]])

In [13]:
torch.stack([x.mean(0), x.std(0), x.min(0).values, x.max(0).values], 1)

tensor([[ 4.5000,  3.8730,  0.0000,  9.0000],
        [ 5.5000,  3.8730,  1.0000, 10.0000],
        [ 6.5000,  3.8730,  2.0000, 11.0000]])

In [18]:
torch.arange(20).reshape(2, 2, 5).swapaxes(1, 2).swapaxes(1, 2) == torch.arange(20).reshape(2, 2, 5)

tensor([[[True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True]]])